In [ ]:
from record import VADRecorder
import librosa
import torch
from models import MFCC1DCNN
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os, sys, warnings

# Suppress ALSA errors
def suppress_alsa_errors():
    sys.stderr = open(os.devnull, 'w')

def restore_stderr():
    sys.stderr = sys.__stderr__

suppress_alsa_errors()

In [ ]:
n_features = 15
model_path = 'logs/best-cnn-mfcc/checkpoints/last.ckpt'

In [ ]:
def to_mfcc(y, n_features = None):
    assert n_features is not None, "specify n_mfcc"
    # Compute MFCC
    mfcc = librosa.feature.mfcc(
        y = y,
        sr=8000,
        n_mfcc=n_features,
        n_fft=256,
        hop_length=80,
        n_mels=40,   
        fmin=20,
        fmax=8000//2
    )
    return mfcc 




device = torch.device('cpu')
model = MFCC1DCNN(n_mfcc=n_features)
checkpoint = torch.load(model_path, map_location='cpu')

if 'state_dict' in checkpoint:
# Lightning checkpoint
    state_dict = {k.replace('model.', ''): v for k, v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
else:
    model.load_state_dict(checkpoint)

model.to(device)
model.eval()

# Live Mic Demo

run the cell to initiate the recording. Once preditction is complete rerun again for new recording and prediction
for best performance, preferably speak right after you are prompted to speak.....

In [ ]:
rec = VADRecorder(aggressiveness=2)

audio_array = rec.record()     # blocks until you finish speaking
print("Recorded NumPy shape:", audio_array.shape)

rec.save("output.wav", audio_array)  # optional
rec.close()
audio_array = audio_array.astype(np.float32) / 32768.0

mfcc = to_mfcc(audio_array, n_features)
mfcc = torch.tensor(mfcc[None,:]).float()
with torch.no_grad():
    logits = model(mfcc)
    pred = torch.argmax(logits, dim=1).item()
    
print("Predicted Digit: ", pred)